In [58]:
import numpy as np
import pandas as pd
from math import floor
mouthday={1:31,2:31,3:28,4:31,5:30,6:31,7:30,8:31,9:31,10:30,11:31,12:30}
def solvetimediff(t2, t1):#这个函数只要在同一个月份就可以计算很正确
    t1sec = t1 % 100
    t2sec = t2 % 100
    t1 = floor(t1 / 100)
    t2 = floor(t2 / 100)
    t1min = t1 % 100
    t2min = t2 % 100
    t1 = floor(t1 / 100)
    t2 = floor(t2 / 100)
    t1hour = t1 % 100
    t2hour = t2 % 100
    t1 = floor(t1 / 100)
    t2 = floor(t2 / 100)
    t1day = t1 % 100
    t2day = t2 % 100
    t1 = floor(t1 / 100)
    t2 = floor(t2 / 100)
    t1mth = t1 % 100
    t2mth = t2 % 100
    if t2sec < t1sec:
        t2min = t2min - 1
        t3sec = t2sec + 60 - t1sec
    else:
        t3sec = t2sec - t1sec
    if t2min < t1min:
        t2hour = t2hour - 1
        t3min = t2min + 60 - t1min
    else:
        t3min = t2min - t1min
    if t2hour < t1hour:
        t2day = t2day - 1
        t3hour = t2hour + 24 - t1hour
    else:
        t3hour = t2hour - t1hour
    if t2day < t1day:
        t2mth = t2mth - 1
        t3day = t2day + mouthday[t2mth] - t1day
    else:
        t3day = t2day - t1day
    return t3sec+t3min*60+t3hour*3600+t3day*86400
def addsecmilediff(df):
    timediff = []
    time = df.time.tolist()
    for index, t in enumerate(time):
        if(t==time[0]):
            timediff.append(0)
            continue
        timediff.append(solvetimediff(t,time[index-1]))
    df['secdiff']=timediff
    return df

In [59]:
import numpy as np
import pandas as pd
testdata1=[]
for number in range(5):
    for fragment in range(10):
        path=r"C:\Users\14020\Desktop\NCBDC 2019\TestData"+'\\'+"Vehicle No."+str(number)+"_Fragment No."+str(fragment)+".csv"
        df=addsecmilediff(pd.read_csv(path))
        index=df.loc[df.total_voltage==0.0].loc[df.total_current==0.0].loc[df.soc==0.0].index
        df=df.drop(index)
        testdata1.append(df)
for number in range(5,15):
    for fragment in range(5):
        path=r"C:\Users\14020\Desktop\NCBDC 2019\TestData"+'\\'+"Vehicle No."+str(number)+"_Fragment No."+str(fragment)+".csv"
        df=addsecmilediff(pd.read_csv(path))
        index=df.loc[df.total_voltage==0.0].loc[df.total_current==0.0].loc[df.soc==0.0].index
        df=df.drop(index)
        testdata1.append(df)

In [60]:
from scipy.interpolate import lagrange
def interp_lagrange(data,p,k=3):#我就看了一些时间差过长但是里程差小于0.3的数据 soc是没有变化的 所以我拉格朗日插值的时候选取前后3个数据作为基函数
    # 调用拉格朗日插值，得到插值函数p
    num=int(data.loc[p].secdiff/10-1)
    index=np.hstack((np.array([i+p-k for i in range(k)]),np.array([i+p+1 for i in range(k)])))
    x=np.hstack((np.array([i for i in range(k)]),np.array([i for i in range(k+num,2*k+num)])))
    xx=np.array([i for i in range(k,k+num)])
    indexdata=data.loc[index]
    total_voltagep=lagrange(x,indexdata.total_voltage.to_numpy())
    total_currentp=lagrange(x,indexdata.total_current.to_numpy())
    socp=lagrange(x,indexdata.soc.to_numpy())
    temp_maxp=lagrange(x,indexdata.temp_max.to_numpy())
    temp_minp=lagrange(x,indexdata.temp_min.to_numpy())
    motor_voltagep=lagrange(x,indexdata.motor_voltage.to_numpy())
    motor_currentp=lagrange(x,indexdata.motor_current.to_numpy())
    new=pd.DataFrame(columns=data.columns)
    for i in range(num):
        new.loc[i]=[20180000000000,total_voltagep(xx)[i],total_currentp(xx)[i],socp(xx)[i],temp_maxp(xx)[i],temp_minp(xx)[i],motor_voltagep(xx)[i],motor_currentp(xx)[i],0,10]
    if len(data)==p:
        data=pd.concat([data.loc[0:p-1],new],ignore_index=True)
    else:
        data=pd.concat([data.loc[0:p-1],new,data.loc[p:]],ignore_index=True)
    return data

In [61]:
for number in range(75):
    df=testdata1[number]
    if(len(df.loc[df.secdiff>10].loc[df.secdiff<100])):
        index=df.loc[df.secdiff>10].loc[df.secdiff<100].index
        size=len(index)
        for i in range(size):
            cutpoint=df.loc[df.secdiff<100].loc[df.secdiff>=20].index[i]
            df=interp_lagrange(df,cutpoint,k=1)
    df.to_csv(r"C:\Users\14020\Desktop\NCBDC 2019\clearTestData"+'\\'+str(number)+".csv")

D:\ANACONDA\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
